In [22]:
#更新dic_company字典表
# !/usr/bin/python
# -*- coding: utf-8 -*-
# Author: Selvaria

import requests
import time
from bs4 import BeautifulSoup
import csv
import pandas
import random
from selenium import webdriver
import pymysql
import concurrent.futures
from DBUtils.PooledDB import PooledDB
import datetime
import json
import re

now = datetime.datetime.now()
today = now.strftime('%Y-%m-%d')
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
DATE_FORMAT = "%m/%d/%Y %H:%M:%S %p"


_dbservice_pool = None

class MysqlPool(object): #设置数据库连接池和初始化    
    """
        MYSQL数据库对象，负责产生数据库连接 , 此类中的连接采用连接池实现
        获取连接对象：conn = Mysql.get_connection()
        释放连接对象;conn.close()或del conn
    """
    
    def __init__(self,mincached=5, maxcached=14,
                maxconnections=244, blocking=True, maxshared=0):
        """
            生成MySQL数据库连接池
        :param mincached: 最少的空闲连接数，如果空闲连接数小于这个数，pool会创建一个新的连接
        :param maxcached: 最大的空闲连接数，如果空闲连接数大于这个数，pool会关闭空闲连接
        :param maxconnections: 最大的连接数
        :param blocking: 当连接数达到最大的连接数时，在请求连接的时候，如果这个值是True，请求连接的程序会一直等待，
                          直到当前连接数小于最大连接数，如果这个值是False，会报错，
        :param maxshared: 当连接数达到这个数，新请求的连接会分享已经分配出去的连接
        """        
        db_config = {
            "host": '47.92.25.70',
            "port": 3306,
            "user": 'root',
            "passwd": 'Wfn031641',
            "db": 'cxd_data',
            "charset": 'utf8'
        }
        self.pool = PooledDB(pymysql, mincached=mincached, maxcached=maxcached, maxconnections=maxconnections,
                             blocking=blocking, maxshared=maxshared, **db_config)

    def get_connection(self):
        return self.pool.connection()

    def close(self):
        self.pool.close()

    def __del__(self):
        self.close()

# 获取mysql数据库服务器的连接
def get_dbservice_mysql_conn(): #实例化连接池类，方便调用
    """
    :return: Object  MySQL Connection
    """
    global _dbservice_pool
    if not _dbservice_pool or not isinstance(_dbservice_pool, MysqlPool):
        _dbservice_pool = MysqlPool()
    return _dbservice_pool.get_connection()


class QInfoAddition(object):
    
    headers = {
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.80 Safari/537.36'
            }
    
    '''def __init__(self, name):
        self.name = name'''
    
    def get_login(self): #利用selenium手动登录,获取cookies
        url = 'https://www.qichacha.com/user_login'
        #CHROME_OPTIONS = webdriver.ChromeOptions()
        #CHROME_OPTIONS.add_argument(proxy)
        #browser = webdriver.Chrome(chrome_options=CHROME_OPTIONS)
        browser = webdriver.Chrome()
        browser.get(url) 
        time.sleep(30) #登录并录入验证码  #13342264161 #18698925437 #15524882680 ldp520615@ #13342270074 #13342270064 #13372864060 #13372864160
        cookies = browser.get_cookies()
        post={} #定义一个空的字典，存放cookies内容
        #获取到的cookies是列表形式，将cookies转成字典格式
        for cookie_item in cookies:
            post[cookie_item['name']] = cookie_item['value'] 
        browser.quit()
        cookie_str = json.dumps(post)
        return post,cookies #第一个是requests用，第二个是selenium用

    def get_company_code(self, cookies, name): #获取企业代号No（企查查id）
        company_name = name
        url_s = requests.get('https://www.qichacha.com/search?key=%s' %company_name, headers = self.headers, cookies = cookies).content
        soup_s = BeautifulSoup(url_s,'html.parser')
        try:
            num_u = soup_s.find('span', attrs = {'id': "countOld"})
            num = num_u.find('span', attrs = {'class': "text-danger"})
        except Exception as e:
            logging.warning('基本被封，请手动查询')
            print(str(e))
            #return None
            self.get_login()
            exit(1) # 有错误退出

        if num and num.get_text().strip() == '0':
            print('无此企业', name)
            return 0
        else:
            time.sleep(1)
            search_url = soup_s.find('a', attrs = {'class': "ma_h1"})['href']
            company_code = search_url[6:-5]
            #url_company = requests.get('https://www.qichacha.com%s' %search_url, headers = self.headers).content
            com_web_q = 'https://www.qichacha.com' + search_url #企查查的企业主页
            return company_code
        
    def get_com_status(self, code, cookies): #基础信息
        url = 'https://www.qichacha.com/firm_' + code +'.html'
        #print(url)
        url_company = requests.get(url, headers = self.headers, cookies = cookies).content
        soup_c = BeautifulSoup(url_company,'html.parser')
        try:
            company_title = soup_c.find('div', attrs = {'class': "row title jk-tip"}).find('h1').get_text() #企业名
            company_lead = soup_c.find('a', attrs = {'class': "bname"}).get_text() #法人
            company_tel_o = soup_c.find('span', attrs = {'class': "cvlu"}).get_text().strip()
            pattern = re.compile(r'\d+') 
            company_tel_list = pattern.findall(company_tel_o)
            if len(company_tel_list) > 0:
                company_tel = company_tel_list[0]
            else:
                company_tel = None            
            company_email = soup_c.find_all('span', attrs = {'class': "cvlu"})[2].get_text().strip()
            #print(company_lead, company_tel, company_email)
            company_table = soup_c.find_all('table', attrs = {'class': "ntable"})[1]
            table = company_table.find_all('td')
            data_sheet = []
            for i in table:
                data = i.get_text().strip()
                data_sheet.append(data)
            #print(len(data_sheet))
            #print(data_sheet)
            dict_d = {}
            for i in range(0,40,2):
                dict_d[data_sheet[i]] = data_sheet[i+1]
            #print(dict_d)
            found_cash = dict_d['注册资本']
            found_date = dict_d['成立日期']
            company_loc = dict_d['企业地址'].split('\n')[0]
            company_cover = dict_d['经营范围']
            company_tax_num = dict_d['纳税人识别号']
            company_prov = dict_d['所属地区']
            #print('公司名称：%s' %company_title)
            msg = [company_prov, company_title, company_lead, found_date, found_cash, company_loc, company_email, company_tel,\
                   company_cover, url, company_tax_num]
            print('%s web data is done.' %code)
            return msg
        except Exception as e:
            print(repr(e))
            print('网页数据提取出现问题')
    
    def get_addition_status(self, code, cookies): #补充信息，这个信息在招商库里
        url_t = 'https://www.qichacha.com/cbase_' + code
        url_l= 'https://www.qichacha.com/user_login'
        CHROME_OPTIONS = webdriver.ChromeOptions()
        CHROME_OPTIONS.add_argument('headless')
        browser = webdriver.Chrome(chrome_options=CHROME_OPTIONS)
        #browser = webdriver.Chrome() 
        browser.get(url_l) 
        for cookie_item in cookies:
            browser.add_cookie(cookie_item)
        browser.get(url_t)
        time.sleep(2)
        try:
            info_0 = browser.find_element_by_xpath('//*[@id="Cominfo"]/table[2]/tbody/tr[2]/td[2]').text #经营状态
        except:
            info_0 = None
        if '在业' in info_0:
            info = 1
        elif '存续' in info_0:
            info = 2
        elif '注销' in info_0:
            info = 10
        elif '吊销' in info_0:
            info = 9
        elif '筹建' in info_0:
            info = 3
        elif '清算' in info_0:
            info = 4
        elif '迁入' in info_0:
            info = 5
        elif '迁出' in info_0:
            info = 6
        elif '停业' in info_0:
            info = 7
        elif '撤销' in info_0:
            info = 8
        elif '开业' in info_0:
            info = 1
        else:
            info = None
        #print(info)
        status_id = str(info)
        login_code = browser.find_element_by_xpath('//*[@id="Cominfo"]/table[2]/tbody/tr[4]/td[2]').text #注册号
        party_code = browser.find_element_by_xpath('//*[@id="Cominfo"]/table[2]/tbody/tr[4]/td[4]').text   #组织机构代码
        industry_part = browser.find_element_by_xpath('//*[@id="Cominfo"]/table[2]/tbody/tr[5]/td[4]').text #所属行业
        login_position = browser.find_element_by_xpath('//*[@id="Cominfo"]/table[2]/tbody/tr[6]/td[4]').text #注册地
        used_name = browser.find_element_by_xpath('//*[@id="Cominfo"]/table[2]/tbody/tr[8]/td[2]').text #曾用名
        employer_amnt = browser.find_element_by_xpath('//*[@id="Cominfo"]/table[2]/tbody/tr[9]/td[2]').text #人员规模
        insurance_num = browser.find_element_by_xpath('//*[@id="Cominfo"]/table[2]/tbody/tr[8]/td[4]').text #参保人数
        runnint_time = browser.find_element_by_xpath('//*[@id="Cominfo"]/table[2]/tbody/tr[9]/td[4]').text #营业期限
        data = [status_id,login_code,party_code,industry_part,login_position,used_name,employer_amnt,insurance_num,runnint_time]
        data_handle = [None if '-' in x and len(x)==1 else x for x in data]
        browser.quit()
        return data_handle
        

def test(name):
    q = QInfoAddition()
    cookies_b, cookies_s = q.get_login()
    company_code = q.get_company_code(cookies_b, name)
    data = q.get_com_status(company_code, cookies_b)
    return data

#test('浙江宏途物流有限公司')
test('中油鑫企源(大连)石油化工有限公司')

4142960f51c7bf3a62be965dba750d31 web data is done.


['辽宁省',
 '中油鑫企源(大连)石油化工有限公司',
 '任大伟',
 '2018-07-09',
 '3000万元人民币',
 '辽宁省大连长兴岛经济区何屯村(原何屯小学)',
 '暂无',
 None,
 '成品油(仅限汽油、煤油、柴油[闭杯闪点≤60℃])、石脑油、石油醚、煤焦油、溶剂油[闭杯闪点≤60℃]、苯乙烯[稳定的]、苯酚、苯、粗苯、重质苯、乙酸乙酯、乙酸正丁酯、1,2-二甲苯、1,3-二甲苯、1,4-二甲苯、二甲苯异构体混合物、乙苯、1,2,3-三甲基苯、1,2,4-三甲基苯、1,3,5-三甲基苯、1,2,4,5-四甲苯、乙烯、丙烯、1-丁烯、2-丁烯、1,3-丁二烯[稳定的]、异丁烯、2-戊烯、环戊烯、2-甲基-2-丁烯、3-甲基-1-丁烯、甲烷、乙烷、丙烷、正丁烷、异丁烷、环丁烷、正戊烷、环戊烷、2-甲基丁烷、2-甲基戊烷、正己烷、异辛烷、1,2-环氧丙烷、环氧乙烷、乙炔、甲基叔丁基醚、甲醇、乙醇[无水]、正丁醇、环己酮、氢氧化钠、煤焦沥青、硝化沥青、三聚丙烯、四聚丙烯、蒽油乳膏、蒽油乳剂、含易燃溶剂的合成树脂、油漆、辅助材料、涂料等制品:[碳四、碳五、碳九、混合芳烃、重芳烃、轻芳烃、稳定轻烃、混合苯、混合二甲苯、重烯烃、杂醇、轻质燃料油、轻质油、汽油调和组分、洗油(81)][闭杯闪点≤60℃](2828)的无储存经营,柴油[闭杯闪点>60℃]、燃料油[闭杯闪点>60℃]、渣油[闭杯闪点>60℃]、重油[闭杯闪点>60℃]、工业白油[闭杯闪点>60℃]、白油[闭杯闪点>60℃]、粗白油[闭杯闪点>60℃]、有机热载体(导热油)[闭杯闪点>60℃]、芳香基矿物油(基础油)[闭杯闪点>60℃]、液压油(变压器油)[闭杯闪点>60℃]、橡胶增塑剂(橡胶及橡胶制品)[闭杯闪点>60℃]、石油焦[闭杯闪点>60℃](以上均不含危险化学品)、蜡油、润滑油、钢材、木材、煤炭、仪器仪表、电线电缆、矿产品、阀门、办公用品、汽车零部件、计算机硬件、建材、化工产品、五金产品销售,城镇燃气经营,经济信息咨询,计算机软件开发,经营广告业务,国内一般贸易,货物进出口、技术进出口。(依法须经批准的项目,经相关部门批准后方可开展经营活动)***(依法须经批准的项目,经相关部门批准后方可开展经营活动。)',
 'https://www.qichacha.com/firm_4

In [4]:
connection = pymysql.connect(host='39.105.9.20', user='root', passwd='bigdata_oil',
                db='cxd_data', port=3306, charset='utf8')
sql_t = "select * from `financial_exchange` where `year` = '2019'"
df_t = pandas.read_sql(sql_t, connection)
#len(df_t)
df_t.head()

,financial_exchange_id,Xf_com_group,Xf_com_type,Xf_company_id,Xf_company_name,Gf_com_group,Gf_com_type,Gf_company_id,Gf_company_name,exchange_date,...,year,exchangetype,main_id,main_name,spec_id,spec_name,good_id,good_name,org_id,add_time
0,1435912,0.0,1.0,4355,山东齐成石油化工有限公司,1.0,2.0,2411,中油东港（大连）能源有限公司,2019-02-28,...,2019,1,None,None,None,None,None,None,None,NaT
1,2331407,1.0,2.0,2568,大连隆泰石油化工有限公司,0.0,1.0,16422,淄博市临淄鲁辉化工有限公司,2019-01-07,...,2019,1,None,None,None,None,None,None,None,2019-05-09 20:11:01
2,2331408,1.0,2.0,2568,大连隆泰石油化工有限公司,0.0,1.0,16422,淄博市临淄鲁辉化工有限公司,2019-01-07,...,2019,1,None,None,None,None,None,None,None,2019-05-09 20:11:01
3,2331409,1.0,2.0,2568,大连隆泰石油化工有限公司,0.0,1.0,16422,淄博市临淄鲁辉化工有限公司,2019-01-07,...,2019,1,None,None,None,None,None,None,None,2019-05-09 20:11:01
4,2331410,1.0,2.0,2568,大连隆泰石油化工有限公司,0.0,1.0,16422,淄博市临淄鲁辉化工有限公司,2019-01-07,...,2019,1,None,None,None,None,None,None,None,2019-05-09 20:11:01


In [5]:
x_middle = df_t['Xf_company_name'].values
x_com = list(set(x_middle))
x_com

['大连道氏贸易有限公司',
 '中国石油运输有限公司宁夏分公司',
 '扬州市江都区新兴加油站',
 '大连君创能源有限公司',
 '黔东南州华明电力服务有限责任公司成品油销售分公司',
 '大连中油兴达石油化工有限公司',
 '北京桔子水晶酒店管理咨询有限公司上海第一分公司',
 '中油利安达（大连）石油化工有限公司',
 '大连汇海石油化工有限公司',
 '中国石化销售有限公司吉林长春石油分公司',
 '郑州海源加油站',
 '通化良友石化加油站',
 '盘锦厚德化工有限公司',
 '大连驿居杏林酒店有限公司',
 '华住酒店管理有限公司淄博分公司',
 '大连百恒化工有限公司',
 '中海联港石化（大连）有限公司',
 '云南云岭石化有限公司岗纪服务区',
 '北京日坛宾馆',
 '大连市中山区博雅电脑图文设计室',
 '铁法煤业（集团）有限责任公司煤炭销售分公司',
 '中油润添（大连）能源有限公司',
 '中国石油天然气股份有限公司广东肇庆销售分公司',
 '大连源惠通石油化工有限公司',
 '中油致远（大连）石油化工有限公司',
 '大连易思特化工有限公司',
 '深茂石油化工（大连）有限公司',
 '中国石油天然气股份有限公司湖北黄冈销售分公司',
 '利津远东交通运输有限公司',
 '湖北交投石化能源发展有限公司',
 '温州市印象加油站有限公司',
 '大连森科信息技术发展有限公司',
 '江苏小米化工贸易有限公司',
 '青岛丽东化工有限公司',
 '同发石油化工（大连）有限公司',
 '郑州市华宿酒店管理有限公司',
 '龙井众诚能源发展有限公司',
 '大连奥莱姆石化有限公司',
 '大连君通石油化工有限公司',
 '北京市顺通加油站',
 '中国石化销售有限公司河北邢台石油分公司',
 '大连新鑫石油化工有限公司',
 '中宁石油（大连）有限公司',
 '中储发展股份有限公司上海吴淞分公司',
 '江西中化石油成品油销售有限公司',
 '中油友旺石油（大连）有限公司',
 '东营市富嘉物流有限责任公司',
 '大连运起石化有限公司',
 '大连鸿利达新材料有限公司',
 '武汉市兴荣汽车运输有限公司',
 '三亚半岭温泉海韵别墅度假酒店有限公司',
 '大连盛鑫源石化有限公司',
 '大连鑫川石油化工有限公司',
 '大连中诚石油化工有限

In [6]:
def check_company(com):
    connection = pymysql.connect(host = '47.92.25.70',user = 'root',password = 'Wfn031641',db = 'cxd_data',charset = 'utf8')
    try:
        with connection.cursor() as cursor: 
            sql = "select `id` from `dic_company` where `name` = '{}'".format(com)
            cursor.execute(sql)
            cover = cursor.fetchone()[0]
        connection.close()
    except:
        cover = False
    return cover

In [15]:
xf_company_new = []
for com in x_com:
    result = check_company(com)
    if result == False:
        xf_company_new.append(com)
    if len(com) < 2:
        continue
xf_company_new

['大连市中山区博雅电脑图文设计室',
 '中油友旺石油（大连）有限公司',
 '-1',
 '中物智联（大连）科技有限公司',
 '中油润拓能源（大连）有限公司',
 '中油弘汇新石油化工（大连）有限公司',
 '华宇（大连）酒店管理有限公司',
 '中油中铝（北京）石油化工有限公司',
 '智龙（大连）商贸有限公司',
 '冷水江市铎山加油站（普通合伙）',
 '维尔安石化（浙江）有限公司',
 '温州公交石油供应中心',
 '雷州市国防交通加油站',
 '中油鑫港石油（大连）有限公司',
 '盛华石油化工（大连）有限公司',
 '中国石化销售有限公司陕西榆林石油分公司',
 '汉能中科（青岛）新材料有限公司',
 '中油众邦能源（大连）有限公司',
 '去哪儿网（天津）国际旅行社有限公司',
 '内蒙古字仁律师事务所',
 '平江县鸿源加油站',
 '甘井子区辛寨子街道都市晨光旅店',
 '七天快捷酒店管理（北京）有限公司鼓楼店',
 '中油久丰石油化工（大连）有限公司',
 '抚顺市望花区乐家快捷酒店',
 '沧县昌盛加油站（普通合伙）',
 '中油恒生石油化工(大连）有限公司',
 '中国石化销售有限公司吉林四平石油分公司',
 '中油旭力（大连）石化有限公司',
 '法塞特酒业（上海）有限公司',
 '中油盛安石油化工（大连）有限公司',
 '瑞中石油（成都）有限公司',
 '中油乾元石油化工(大连）有限公司',
 '中石化森美（福建）石油有限公司漳州分公司',
 '中海晖泰石油化工（大连）有限公司',
 '中油坤源能源（大连）有限公司',
 '驰顺晟通供应链管理（大连）有限公司',
 '中国石化销售有限公司河南焦作石油分公司',
 '中油鼎新石油化工（大连)有限公司',
 '中油润泽嘉华石化（大连）有限公司',
 '中油三维丝石化（大连）有限公司',
 '中油中顺石油化工（大连）有限公司',
 '泊朗威门窗（大连）有限公司',
 '新乡市平原示范区祥瑞商务温泉快捷酒店',
 '中油中广石油化工（大连）有限公司',
 '中能盛世方成石油化工（大连）有限公司',
 '中国石油天然气股份有限公司甘肃武威销售分公司武威南服务区加油站（母站）',
 '六通能源（大连）有限公司',
 '六安市舒城县舒粮加油站（普通合伙）',
 '中汇宝路能源（大连）有限公司',


In [16]:
print(len(x_com))
print(len(xf_company_new))

2159
196


In [17]:
g_middle = df_t['Gf_company_name'].values
g_com = list(set(g_middle))
gf_company_new = []
for com in g_com:
    result = check_company(com)
    if result == False:
        gf_company_new.append(com)
    elif len(com) < 2:
        continue
    else:
        continue
print(gf_company_new)
print(len(g_com))
print(len(gf_company_new))

['', '铜陵市鑫来物流有限公司', '中铁北京工程局集团（天津）工程有限公司', '中石化中海船舶燃料供应辽宁有限公司', '哈尔滨市西岗区红旗满族乡人民政府', '遵义市建安混凝土运输有限公司', '米拉美亚（大连）国际贸易有限公司', '东莞市沙田裕丰塑胶制品厂', ' 瓦房店中德瑞孚轴承制造有限公司', '中图', '济南百斯杰生物工程有限公司\t', '南宁市志德港埠有限公', '候美冬', '山东东方华龙工贸集团有限公司\t', '贵州同润能源有限公司\t', '山东寿光健元春化工有限公司\t\n', '苏州兴昌润化工有限公司\t', '大鹿岛贝类养殖有限公司', '中油聖业（大连）能源有限公司', '青岛中惠乔雨工贸有限公司', '广东省四会监狱', '唐河油田中微化工厂', '广东省江门监狱', '大连友谊（集团）股份有限公司友谊商城开发区店', '四川华诺石油化工有限公司', '重庆风驰驿达物流有限公司', '济南鑫创意化工有限公司\t', '东营大地硅业有限公司\t', '中油海能（大连）石油化工有限公司', '遂昌县焦滩加油站', '郑州优尼凯贸易有限公司\t\t\t', '汤始建华建材（淮安）有限公司', '陈旗宝镇鑫金宝运输车队', '金港丰石油化工（大连）有限公司', '米阳物流科技（宁波）有限公司', '大石桥唯鸿运输有限公司', '楷翼国际物流（上海）有限公司', '湖北格润化工有限公司\t', '天津开发区鸿运达建筑工程有限公司', '建华建材（中山）有限公司', '济南永瑞化工有限公司\t', '杭州联力建设有限公司', '丹东东港经济开发区物业有限公司六合加油站', '介休市城区旭日装卸服务部', '捷尔（舟山）能源有限公司', '信阳市河源石油有限公司', '中油双凯石油销售（大连）有限公司', '哈尔滨市第五医院', '中油中铝大连石油化工有限公司', '长沙阳鸿机械化租赁有限公司\t', '焦作万方铝业股份有限公司', '哈尔滨市南岗区红旗满族乡人民政府', '吉林达通公路工程有限公司', '河北驰野玻璃珠有限公司', '宝洁（中国）营销有限公司', '恒力石化（大连）有限公司', '山东奔腾漆业股份有限公司', '恒力石化（大连）炼化有限公司', '广饶县华星小李加油站（普通合伙）', '合肥承影供应链管理有限公司',

In [19]:
up_com_list = gf_company_new + xf_company_new
print(len(up_com_list))
up_com_list

554


['',
 '铜陵市鑫来物流有限公司',
 '中铁北京工程局集团（天津）工程有限公司',
 '中石化中海船舶燃料供应辽宁有限公司',
 '哈尔滨市西岗区红旗满族乡人民政府',
 '遵义市建安混凝土运输有限公司',
 '米拉美亚（大连）国际贸易有限公司',
 '东莞市沙田裕丰塑胶制品厂',
 ' 瓦房店中德瑞孚轴承制造有限公司',
 '中图',
 '济南百斯杰生物工程有限公司\t',
 '南宁市志德港埠有限公',
 '候美冬',
 '山东东方华龙工贸集团有限公司\t',
 '贵州同润能源有限公司\t',
 '山东寿光健元春化工有限公司\t\n',
 '苏州兴昌润化工有限公司\t',
 '大鹿岛贝类养殖有限公司',
 '中油聖业（大连）能源有限公司',
 '青岛中惠乔雨工贸有限公司',
 '广东省四会监狱',
 '唐河油田中微化工厂',
 '广东省江门监狱',
 '大连友谊（集团）股份有限公司友谊商城开发区店',
 '四川华诺石油化工有限公司',
 '重庆风驰驿达物流有限公司',
 '济南鑫创意化工有限公司\t',
 '东营大地硅业有限公司\t',
 '中油海能（大连）石油化工有限公司',
 '遂昌县焦滩加油站',
 '郑州优尼凯贸易有限公司\t\t\t',
 '汤始建华建材（淮安）有限公司',
 '陈旗宝镇鑫金宝运输车队',
 '金港丰石油化工（大连）有限公司',
 '米阳物流科技（宁波）有限公司',
 '大石桥唯鸿运输有限公司',
 '楷翼国际物流（上海）有限公司',
 '湖北格润化工有限公司\t',
 '天津开发区鸿运达建筑工程有限公司',
 '建华建材（中山）有限公司',
 '济南永瑞化工有限公司\t',
 '杭州联力建设有限公司',
 '丹东东港经济开发区物业有限公司六合加油站',
 '介休市城区旭日装卸服务部',
 '捷尔（舟山）能源有限公司',
 '信阳市河源石油有限公司',
 '中油双凯石油销售（大连）有限公司',
 '哈尔滨市第五医院',
 '中油中铝大连石油化工有限公司',
 '长沙阳鸿机械化租赁有限公司\t',
 '焦作万方铝业股份有限公司',
 '哈尔滨市南岗区红旗满族乡人民政府',
 '吉林达通公路工程有限公司',
 '河北驰野玻璃珠有限公司',
 '宝洁（中国）营销有限公司',
 '恒力石化（大连）有限公司',
 '山东奔腾漆业股份有限公司'

In [24]:
for i in up_com_list:
    if len(i) < 5:
        up_com_list.remove(i)
    else:
        pass
    
up_com_list

['铜陵市鑫来物流有限公司',
 '中铁北京工程局集团（天津）工程有限公司',
 '中石化中海船舶燃料供应辽宁有限公司',
 '哈尔滨市西岗区红旗满族乡人民政府',
 '遵义市建安混凝土运输有限公司',
 '米拉美亚（大连）国际贸易有限公司',
 '东莞市沙田裕丰塑胶制品厂',
 ' 瓦房店中德瑞孚轴承制造有限公司',
 '济南百斯杰生物工程有限公司\t',
 '南宁市志德港埠有限公',
 '山东东方华龙工贸集团有限公司\t',
 '贵州同润能源有限公司\t',
 '山东寿光健元春化工有限公司\t\n',
 '苏州兴昌润化工有限公司\t',
 '大鹿岛贝类养殖有限公司',
 '中油聖业（大连）能源有限公司',
 '青岛中惠乔雨工贸有限公司',
 '广东省四会监狱',
 '唐河油田中微化工厂',
 '广东省江门监狱',
 '大连友谊（集团）股份有限公司友谊商城开发区店',
 '四川华诺石油化工有限公司',
 '重庆风驰驿达物流有限公司',
 '济南鑫创意化工有限公司\t',
 '东营大地硅业有限公司\t',
 '中油海能（大连）石油化工有限公司',
 '遂昌县焦滩加油站',
 '郑州优尼凯贸易有限公司\t\t\t',
 '汤始建华建材（淮安）有限公司',
 '陈旗宝镇鑫金宝运输车队',
 '金港丰石油化工（大连）有限公司',
 '米阳物流科技（宁波）有限公司',
 '大石桥唯鸿运输有限公司',
 '楷翼国际物流（上海）有限公司',
 '湖北格润化工有限公司\t',
 '天津开发区鸿运达建筑工程有限公司',
 '建华建材（中山）有限公司',
 '济南永瑞化工有限公司\t',
 '杭州联力建设有限公司',
 '丹东东港经济开发区物业有限公司六合加油站',
 '介休市城区旭日装卸服务部',
 '捷尔（舟山）能源有限公司',
 '信阳市河源石油有限公司',
 '中油双凯石油销售（大连）有限公司',
 '哈尔滨市第五医院',
 '中油中铝大连石油化工有限公司',
 '长沙阳鸿机械化租赁有限公司\t',
 '焦作万方铝业股份有限公司',
 '哈尔滨市南岗区红旗满族乡人民政府',
 '吉林达通公路工程有限公司',
 '河北驰野玻璃珠有限公司',
 '宝洁（中国）营销有限公司',
 '恒力石化（大连）有限公司',
 '山东奔腾漆业股份有限公司',
 '恒力石化（大连）炼化有限公司',

In [21]:
id_list =[]
for com in up_com_list[1:]:
    com_id = check_company(com)
    if com_id == False:
        continue
    else:
        id_list.append(com_id)
        
print(id_list)
print(len(id_list))

[]
0


In [25]:
def up_sql_company(data): #上传数据库dic_company
    connection = get_dbservice_mysql_conn()
    if '长兴岛' in data[5]:
        try :
            with connection.cursor() as cursor:
                sql_i = "INSERT INTO `dic_company` (`name`,`com_group`,`company_province`,`company_provinceId`,`company_city`,`company_cityId`)\
                VALUES(%s,%s,%s,%s,%s,%s)"
                cursor.execute(sql_i,[data[1], 1, data[0], 8, '大连市', 51])
            connection.commit() 
            #print('成功')
        except pymysql.err.IntegrityError as e:
            print('dic_company出现重复值，进行覆盖',e)
        except Exception as e:
            print(e)
    else:
        try :
            with connection.cursor() as cursor:
                sql_i = "INSERT INTO `dic_company` (`name`,`com_group`,`company_province`) VALUES(%s,%s,%s)"
                cursor.execute(sql_i,[data[1], 0, data[0]])
            connection.commit() 
            #print('成功')
        except  pymysql.err.IntegrityError as e:
            print('dic_company出现重复值，进行覆盖',e)
            print(e)
        finally:
            connection.close()
    print('"dic_company" Single is Done')
    
def get_id_data(data):
    connection = get_dbservice_mysql_conn()
    with connection.cursor() as cursor: 
        sql = "select `id` from `dic_company` where `name` = '{}'".format(data[1])
        cursor.execute(sql)
        c_id = cursor.fetchone()[0]
    connection.close()
    data.pop(0)
    data.insert(-1, c_id)
    return data

def up_sql_detail(data): #上传数据库dic_company_detail
    connection = get_dbservice_mysql_conn()
    try :
        with connection.cursor() as cursor:
            sql_i = "insert into `dic_company_detail`(`name`,`person`,`date`,`capital`,`address`,`email`,`phone`,`operate`,`website`,\
            `company_id`, `tax_num`)\
            values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql_i,data)
        connection.commit() 
        #print('成功')
    except Exception as e:
        print('company_detail出现重复值，进行覆盖',e)
        print(e)
    finally:
        connection.close()
    print('"dic_company_detail" Single is Done')

In [26]:
test_com = up_com_list[1]
#print(test_com)

q = QInfoAddition()
cookies_b, cookies_s = q.get_login()

error_list = []

for com in up_com_list[:]:
    com_index = up_com_list.index(com)
    company_code = q.get_company_code(cookies_b, com)
    print('开始进行"%s"公司，是第%s个' %(com,com_index))
    try:
        data = q.get_com_status(company_code, cookies_b)
        up_sql_company(data)
    except Exception as e:
        print('%s出现问题' %com)
        print(str(e))
        error_list.append(com)
        continue
    data_detail = get_id_data(data)
    up_sql_detail(data)
    time.sleep(14)

开始进行"铜陵市鑫来物流有限公司"公司，是第0个
6a9104623105ca91daaea296cd2eadf5 web data is done.
(1062, "Duplicate entry '铜陵市鑫来物流园有限公司' for key 'name'")
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"中铁北京工程局集团（天津）工程有限公司"公司，是第1个
5fc12c92d21fbead704a7f3cc698cb68 web data is done.
(1062, "Duplicate entry '中铁北京工程局集团(天津)工程有限公司' for key 'name'")
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"中石化中海船舶燃料供应辽宁有限公司"公司，是第2个
2751380694ae895b18d72fdbd9e9e7e2 web data is done.
(1062, "Duplicate entry '中石化中海船舶燃料供应有限公司辽宁分公司' for key 'name'")
"dic_company" Single is Done
"dic_company_detail" Single is Done
无此企业 哈尔滨市西岗区红旗满族乡人民政府
开始进行"哈尔滨市西岗区红旗满族乡人民政府"公司，是第3个
哈尔滨市西岗区红旗满族乡人民政府出现问题
must be str, not int
开始进行"遵义市建安混凝土运输有限公司"公司，是第4个
09a9b64bf2dc10986911aa021f9045c0 web data is done.
(1062, "Duplicate entry '遵义市建安混凝土运输服务有限公司' for key 'name'")
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"米拉美亚（大连）国际贸易有限公司"公司，是第5个
80afb652c3517f04cf76a26bbdd5faf1 web data is done.
(

D:\Selvaria\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'operate' at row 1")
  result = self._query(query)


"dic_company_detail" Single is Done
开始进行"贵州同润能源有限公司	"公司，是第11个
810a34937267bf2f19ff605313abf91c web data is done.
(1062, "Duplicate entry '贵州同润能源有限公司' for key 'name'")
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"山东寿光健元春化工有限公司	
"公司，是第12个
dc8dbf1549561ee20377ab8e75b6ae5c web data is done.
(1062, "Duplicate entry '山东寿光健元春化工有限公司' for key 'name'")
"dic_company" Single is Done
"dic_company_detail" Single is Done


KeyboardInterrupt: 

In [67]:
import re

text_tel = '更多号码'

pattern = re.compile(r'\d+') #寻找任意长度整数
price = pattern.findall(text_tel)
price

[]